In [2]:
!pip install --upgrade pip setuptools wheel
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.44.0
    Uninstalling wheel-0.44.0:
      Successfully uninstalled wheel-0.44.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)


In [7]:
import os
from pathlib import Path
import stat

# 1) Create the .kaggle folder in your home dir (works on Windows too)
home = Path.home()
kaggle_dir = home / ".kaggle"
kaggle_dir.mkdir(exist_ok=True)

# 2) Move kaggle.json into place
src = Path("kaggle.json")
dst = kaggle_dir / "kaggle.json"
if not src.exists():
    raise FileNotFoundError("Could not find kaggle.json in the current folder")
src.replace(dst)

WindowsPath('C:/Users/baoph/.kaggle/kaggle.json')

In [8]:
from kaggle.api.kaggle_api_extended import KaggleApi
from pathlib import Path

# 1) authenticate
api = KaggleApi()
api.authenticate()

# 2) ensure a local data/ folder exists
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

# 3) download & unzip
api.dataset_download_files(
    "grassknoted/asl-alphabet",
    path=str(data_dir),
    unzip=True
)
print("Download complete. Files are in", data_dir.resolve())

Dataset URL: https://www.kaggle.com/datasets/grassknoted/asl-alphabet
Download complete. Files are in C:\Users\baoph\Documents\Duovoice\duovoice\model\data


In [9]:
# Inspect the contents of your `data/` folder
import os

for root, dirs, files in os.walk("data"):
    print(f"\nDirectory: {root}")
    if dirs:
        print(" Subfolders:", dirs)
    if files:
        print(" Files (first 10):", files[:10])



Directory: data
 Subfolders: ['asl_alphabet_test', 'asl_alphabet_train']

Directory: data\asl_alphabet_test
 Subfolders: ['asl_alphabet_test']

Directory: data\asl_alphabet_test\asl_alphabet_test
 Files (first 10): ['A_test.jpg', 'B_test.jpg', 'C_test.jpg', 'D_test.jpg', 'E_test.jpg', 'F_test.jpg', 'G_test.jpg', 'H_test.jpg', 'I_test.jpg', 'J_test.jpg']

Directory: data\asl_alphabet_train
 Subfolders: ['asl_alphabet_train']

Directory: data\asl_alphabet_train\asl_alphabet_train
 Subfolders: ['A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'nothing', 'O', 'P', 'Q', 'R', 'S', 'space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

Directory: data\asl_alphabet_train\asl_alphabet_train\A
 Files (first 10): ['A1.jpg', 'A10.jpg', 'A100.jpg', 'A1000.jpg', 'A1001.jpg', 'A1002.jpg', 'A1003.jpg', 'A1004.jpg', 'A1005.jpg', 'A1006.jpg']

Directory: data\asl_alphabet_train\asl_alphabet_train\B
 Files (first 10): ['B1.jpg', 'B10.jpg', 'B100.jpg', 'B1000.jpg', 'B1001.jpg', 'B1002.j

In [10]:
# ── Cell 3: Extract A–Z plus ‘del’ & ‘space’ landmarks ─────────────────────────

import os, cv2, numpy as np
import mediapipe as mp
from tqdm import tqdm

DATA_DIR = "data/asl_alphabet_train/asl_alphabet_train"
if not os.path.isdir(DATA_DIR):
    raise RuntimeError(f"Train directory not found: {DATA_DIR}")

# Only include A–Z, or exactly 'del' or 'space'
valid_labels = set(list("ABCDEFGHIJKLMNOPQRSTUVWXYZ") + ["del", "space"])

all_imgs = []
for label in sorted(os.listdir(DATA_DIR)):
    if label not in valid_labels:
        continue
    folder = os.path.join(DATA_DIR, label)
    for fn in os.listdir(folder):
        if fn.lower().endswith(".jpg"):
            all_imgs.append((label, os.path.join(folder, fn)))

print(f"Found {len(all_imgs)} images across {len(valid_labels)} classes.")

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

X, y = [], []
for label, img_path in tqdm(all_imgs, desc="Extracting landmarks"):
    img = cv2.imread(img_path)
    if img is None: continue
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = hands.process(img_rgb)
    if not res.multi_hand_landmarks: continue

    lm = res.multi_hand_landmarks[0]
    coords = []
    for p in lm.landmark:
        coords += [p.x, p.y, p.z]
    X.append(coords)
    y.append(label)

hands.close()

X = np.array(X, dtype=np.float32)
y = np.array(y)
print(f"Extracted landmarks for {len(X)} images (skipped {len(all_imgs)-len(X)}).")


Found 84000 images across 28 classes.


Extracting landmarks: 100%|██████████| 84000/84000 [1:05:42<00:00, 21.31it/s]


Extracted landmarks for 63673 images (skipped 20327).


In [12]:
# ── Cell 4: Encode labels & split data ─────────────────────────────────────────
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
y_int = le.fit_transform(y)  # maps 'A'→0, …, 'Z'→25

X_train, X_val, y_train, y_val = train_test_split(
    X, y_int,
    test_size=0.2,
    stratify=y_int,
    random_state=42
)

print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)


Training set: (50938, 63) (50938,)
Validation set: (12735, 63) (12735,)


In [13]:
# ── Cell 5: Build & train the MLP with dynamic output size ────────────────────
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 1) Encode labels and discover number of classes
le = LabelEncoder()
y_int = le.fit_transform(y)        # maps your strings → integers 0…27
num_classes = len(le.classes_)     # should be 28
print("Classes found:", le.classes_)

# 2) Split into train/val
X_train, X_val, y_train, y_val = train_test_split(
    X, y_int,
    test_size=0.2,
    stratify=y_int,
    random_state=42
)
print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape,   y_val.shape)

# 3) Build your MLP with the right number of outputs
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(63,)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(num_classes, activation="softmax"),
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# 4) Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
)


Classes found: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z' 'del' 'space']
Train: (50938, 63) (50938,)
Val:   (12735, 63) (12735,)
Epoch 1/50
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.4272 - loss: 2.0370 - val_accuracy: 0.8925 - val_loss: 0.4133
Epoch 2/50
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8701 - loss: 0.4551 - val_accuracy: 0.9484 - val_loss: 0.2269
Epoch 3/50
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9190 - loss: 0.2933 - val_accuracy: 0.9631 - val_loss: 0.1662
Epoch 4/50
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9369 - loss: 0.2218 - val_accuracy: 0.9529 - val_loss: 0.1501
Epoch 5/50
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9456 - loss: 0.1918 - val_accuracy: 0.9723 - val_loss: 0.1181
Epoch 6/50
1592/1592 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9527 - loss: 0.1672 - val_accuracy: 0.9714 - val_loss: 0.1102
Epoch 7/50
1592/1592 ━━━━━━━

In [15]:
model.save("asl_alphabet_mlp.h5")

import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

print("Saved: asl_alphabet_mlp.h5  and  label_encoder.pkl")


Saved: asl_alphabet_mlp.h5  and  label_encoder.pkl


In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open("asl_alphabet_full.tflite", "wb") as f:
    f.write(tflite_model)